This sheet takes zip code demographic data from 2000, 2010, and 2018(ACS5) and calculates percent change.  
In order for this to work, the 2000_scrape, 2010_scrape, and acs5_2018_api_zip notebooks must be run and the output exported to excel.   
Check that the file paths are correct for you.

In [2]:
import pandas as pd


In [41]:
df00 = pd.read_excel("./excel/2000_scrape.xlsx", dtype={'zipcode': object})
df10 = pd.read_excel("./excel/2010_scrape.xlsx", dtype={'zipcode': object})
df18 = pd.read_excel("./acs5_data/acs5_2018_zip.xlsx", dtype={'zipcode': object})

Before we can concatonate and then sort_values to format it as panel data, df18 must be modified to look like the other 2.

In [42]:
df18 = df18[['zipcode', 'territory', 'town', 'county', 'year', 'pop', 
         'hhold', 'avg_hhold', 'med_age', '18plus', '65plus' ]]


In [53]:
#concat combines them by rows
df = pd.concat([df00, df10, df18])

#panel data format, year1 year2 year1 year 2
df.sort_values(by=['zipcode', 'year'], inplace=True)

Now we will calculate percent change.  
There could be a loop here... but it's only six lines so...

In [54]:
dfmod = df
dfmod['p_pop'] = dfmod.groupby(['zipcode'])['pop'].pct_change()
dfmod['p_hhold'] = dfmod.groupby(['zipcode'])['hhold'].pct_change()
dfmod['p_avg_hhold'] = dfmod.groupby(['zipcode'])['avg_hhold'].pct_change()
dfmod['p_med_age'] = dfmod.groupby(['zipcode'])['med_age'].pct_change()
dfmod['p_18plus'] = dfmod.groupby(['zipcode'])['18plus'].pct_change()
dfmod['p_65plus'] = dfmod.groupby(['zipcode'])['65plus'].pct_change()

In [56]:
dfmod = dfmod[['zipcode', 'territory', 'town', 'county', 'year', 'pop', 'p_pop', 
         'hhold', 'p_hhold', 'avg_hhold', 'p_avg_hhold', 'med_age', 'p_med_age', '18plus', 'p_18plus', '65plus', 'p_65plus']]

In [57]:
dfmod.head()

,zipcode,territory,town,county,year,pop,p_pop,hhold,p_hhold,avg_hhold,p_avg_hhold,med_age,p_med_age,18plus,p_18plus,65plus,p_65plus
0,07059,Warren,Warren,Somerset,2000,14219,NaN,4616,NaN,3.05,NaN,39.1,NaN,9994,NaN,1592,NaN
0,07059,Warren,Warren,Somerset,2010,15311,0.076799,5059,0.095971,3.01,-0.013115,43.8,0.120205,11051,0.105763,2059,0.293342
0,07059,Warren,Warren,Somerset,2018,15783,0.030828,5153,0.018581,3.04,0.009967,46.6,0.063927,12171,0.101348,2832,0.375425
1,07060,Warren,Plainfield,Union,2000,49040,NaN,16161,NaN,2.96,NaN,32.4,NaN,36317,NaN,4868,NaN
1,07060,Warren,Plainfield,Union,2010,44998,-0.082423,14304,-0.114906,3.09,0.043919,33.6,0.037037,33850,-0.067930,3990,-0.180362


I set the index to zipcode just so it looks nicer in excel, you don't have to do this

In [60]:
dfmod.set_index(['zipcode'], inplace=True)
dfmod.to_excel('./excel/demographic_changes.xlsx')